# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [ ]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [9]:
from typing import List, Dict, Tuple
from collections import defaultdict, Counter

def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)

    for sentence in train_text:
        temp = sentence.split()

        if bos:
            temp = ["<s>"] + temp
        if eos:
            temp += ["</s>"]

        for step in range(1, min(len(temp) + 1, order + 1)):
            for ind in range(len(temp) - step + 1):
                ngram = tuple(temp[ind:ind + step])
                ngrams[ngram] += 1

    return dict(ngrams)

In [10]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
            ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [11]:
from typing import Dict, Tuple

def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # Подсчитываем вероятность n-граммы с использованием сглаживания Лапласа
    # counts - словарь с частотами n-грамм
    # V - размер словаря (уникальных токенов), если не указан, будет вычислен автоматически
    # k - параметр сглаживания Лапласа

    if V is None:
        V = len(set(t for ngram in counts.keys() for t in ngram))

    # Добавляем сглаживание Лапласа
    numerator = counts.get(ngram, 0) + k
    denominator = sum(counts[tuple(gram)] for gram in counts.keys() if gram[:-1] == ngram[:-1]) + k * V

    # Вычисляем вероятность
    prob = numerator / denominator

    return prob

In [12]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [203]:
import re

# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None

        self.N = 0
    
    @property
    def V(self) -> int:
        return len(self.vocab)

    def fit(self, train_text: List[str]) -> None:
        # Подсчет vocab и ngrams_count по обучающей выборке
        self.ngrams_count = count_ngrams(train_text)
        self.vocab = [gram for gram in self.ngrams_count.keys() if len(gram) == 1]

    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # считаем логарифм вероятности конкретной нграмы
        proba = calculate_ngram_prob(ngram, self.ngrams_count, self.V, self.k)
        return np.log(proba)

    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']

        words = [ word for word in words if word.strip() != "" ]
        
        logprob = 0
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки

        start = 1 - self.order
        end = 0

        while end < len(words):
            actual_start = start if start > 0 else 0
            actual_end = end + 1

            ngram = tuple(words[actual_start:actual_end])

            logprob += self.predict_ngram_log_proba(ngram)

            start += 1
            end += 1

            self.N += 1

        return logprob

    def ppl(self, text: List[str]) -> float:
        self.N = 0
        
        total_log_proba = 0
        total_ngrams_count = 0

        for line in text:
            tokens = re.split(r"\s+", line.strip())
            log_proba = self.predict_log_proba(tokens)
            total_log_proba += log_proba

        perplexity = np.exp(-total_log_proba / self.N) if self.N > 0 else 0.0

        return perplexity

In [204]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"

    print("Test 2.a passed")

test_lm()

Test 2.a passed


# 2. Предсказания с помощью языковой модели (4 балла)

In [208]:
def predict_next_word(lm, prefix: List[str], topk=4):
    # Предсказывает topk наиболее вероятных продолжений фразы prefix

    probabilities = []

    for word in lm.vocab:
        ngram = tuple(prefix + [word])
        proba = lm.predict_ngram_log_proba(ngram)

        probabilities.append({
            "ngram": ngram,
            "proba": proba,
        })

    probabilities.sort(key=lambda row: row["proba"], reverse=True)

    next_words = [(row["ngram"][-1], row["proba"]) for row in probabilities[:topk]]

    return next_words

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

In [ ]:
#Your code here     

In [209]:
train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
              "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
              "мы сэкономим уйму времени если я сейчас сойду с ума по-моему",'практическое сентября',
              "второе практическое занятие пройдет в офлайне тридцать второго сентября в двенадцать часов тридцать две минуты",
              "в офлайне в тридцать два двенадцать"]

In [210]:
lm = NgramLM(order=2)
lm.fit(train_data)

print("по-моему", predict_next_word(lm, ["по-моему"]))
print("сэкономим", predict_next_word(lm, ["сэкономим"]))
print("нграм", predict_next_word(lm, ["нграм"]))
print("вероятности", predict_next_word(lm, ["вероятности"]))

по-моему [(('<s>',), -3.4339872044851463), (('по-моему',), -3.4339872044851463), (('мы',), -3.4339872044851463), (('сэкономим',), -3.4339872044851463)]
сэкономим [(('<s>',), -3.4339872044851463), (('по-моему',), -3.4339872044851463), (('мы',), -3.4339872044851463), (('сэкономим',), -3.4339872044851463)]
нграм [(('<s>',), -3.332204510175204), (('по-моему',), -3.332204510175204), (('мы',), -3.332204510175204), (('сэкономим',), -3.332204510175204)]
вероятности [(('<s>',), -3.332204510175204), (('по-моему',), -3.332204510175204), (('мы',), -3.332204510175204), (('сэкономим',), -3.332204510175204)]
